<a href="https://colab.research.google.com/github/chekfung/cross_layer_final_project/blob/main/training_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source Code

## Imports Needed Throughout the Project

In [50]:
# All Imports 
import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from google.colab import files

# argument parser
import easydict

## Get and Download Datasets

In [51]:
# MNIST Dataset (Images and Labels)
train_set = dsets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

test_set = dsets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

## More Helper Code for Training and Testing Accuracy

In [52]:
def fuse_conv_bn(model,out_model):

  conv_layer = None
  count = 0

  # 1. for loop to collect all Conv layers
  # 2. for loop to collect all BatchNorm layers
  for layer in model.modules():
    
    if isinstance(layer, nn.BatchNorm2d):

      conv_size = conv_layer.weight.size()


      with torch.no_grad():
        for i in range(conv_size[0]):

          
          # get the conv2d weights
          weights = conv_layer.weight[i]


          denominator = torch.sqrt(layer.eps+layer.running_var)
          gamma = layer.weight[i]
          beta = layer.bias[i]
          

          for j in range(conv_size[1]):
            for k in range(conv_size[2]):
              for l in range(conv_size[3]):
                # update out_model layer[count]

                if count == 0:
                  out_model.conv1.weight[i][j][k][l] = gamma * conv_layer.weight[i][j][k][l]  / denominator[i]  
                else:
                  out_model.conv2.weight[i][j][k][l] = gamma * conv_layer.weight[i][j][k][l]  / denominator[i]  

          # In i loop for bias since only 1D
          if count == 0:
            out_model.conv1.bias[i] = (gamma * (conv_layer.bias[i] - layer.running_mean[i])  / denominator[i]) + beta
          else:
            out_model.conv2.bias[i] = (gamma * (conv_layer.bias[i] - layer.running_mean[i])  / denominator[i]) + beta

        count += 1

    conv_layer = layer # conv2d


## FP32 Model 

In [53]:
class MyConvNet_FP32(nn.Module):
    def __init__(self, args):
        super(MyConvNet_FP32, self).__init__()
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1   = nn.BatchNorm2d(16)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(32)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # Layer 3
        self.lin2  = nn.Linear(7*7*32, 10)

    def forward(self, x):
        # Layer 1
        c1 = self.conv1(x)
        b1  = self.bn1(c1)
        a1  = self.act1(b1)
        p1  = self.pool1(a1)

        # Layer 2
        c2  = self.conv2(p1)
        b2  = self.bn2(c2)
        a2  = self.act2(b2)
        p2  = self.pool2(a2)

        # Flatten and Layer 3
        flt = p2.view(p2.size(0), -1)
        out = self.lin2(flt)
        return out
  
# model = MyConvNet(args)
# model = model.cuda()

## Quantization Helper Code

In [54]:
def simple_quantize_val(val, scale_factor, min_val, max_val):
  value = torch.round(val / scale_factor)

  if (value < min_val):
    value = min_val

  if (value > max_val):
    value = max_val

  return (value * scale_factor)

def fixed_point_quantize_val(val, num_bits, fractional_bits):
  integer_bits = num_bits - fractional_bits - 1 # Subtract one for sign bit
  smallest_step_size = 1 / np.power(2, fractional_bits)
  largest_number = (np.power(2, integer_bits) - 1) + ((np.power(2, fractional_bits)-1) * smallest_step_size)
  smallest_number = -1 * np.power(2, integer_bits)

  value = torch.round(val / smallest_step_size) * smallest_step_size

  if (value < smallest_number):
    value = smallest_number

  if (value > largest_number):
    value = largest_number

  return value

# Perhaps slightly optimized version?
def fixed_point_quantize_faster(val, smallest_step_size, largest_number, smallest_number):

  # Perform Pseudo Quantization
  value = torch.round(val / smallest_step_size) * smallest_step_size

  # Clamp Values
  if (value < smallest_number):
    value = smallest_number

  if (value > largest_number):
    value = largest_number

  return value

### NOTE: ONLY USE THIS SHIT ###
def optimized_tensor_fp_quantize(tens, step_size, largest_num, smallest_num):
  # Do everything in tensor operations
  new_tensor = torch.round(tens / step_size) * step_size
  torch.clamp(new_tensor, min=smallest_num, max=largest_num) 
  return new_tensor

# Right now, only for integer quantization.
def quantize_fp_model_weights(model):
  count = 0
  for layer in model.modules():
    if not isinstance(layer, (nn.ReLU, nn.MaxPool2d)) and count != 0:
        
      with torch.no_grad():
        layer.weight.data = optimized_tensor_fp_quantize(layer.weight.data, model.weights_step_size, model.weights_largest_num_representable, model.weights_smallest_num_representable)

    count += 1
  
  return 0

def quantize_fp_model_biases(model):
  count = 0
  for layer in model.modules():
    if not isinstance(layer, (nn.ReLU, nn.MaxPool2d)) and count != 0:
        
      with torch.no_grad():
        layer.bias.data = optimized_tensor_fp_quantize(layer.bias.data, model.bias_step_size, model.bias_largest_num_representable, model.bias_smallest_num_representable)

    count += 1
  
  return 0

In [55]:
def train_model(arg, model, criterion, optimizer, train_loader, quantize=False):
    print("---Training started")
    # Training the Model
    for epoch in range(arg.epochs):
        for i, (images, labels) in enumerate(train_loader):
            # Load Images into GPU
            images = images.cuda()
            labels = Variable(labels).cuda()

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            L1norm = model.parameters()
            arr = []

            # Calculate L1 Norm (if included in hyperparameters)
            if arg.L1norm == True:
                for name,param in model.named_parameters():
                    if 'weight' in name.split('.'):
                        arr.append(param)

                L1loss = 0
                for Losstmp in arr:
                    L1loss = L1loss+Losstmp.abs().mean()

                if len(arr) > 0:
                    loss = loss+L1loss/len(arr)

            if quantize:
                # quantize loss
                loss.data = optimized_tensor_fp_quantize(loss.data, model.loss_step_size, model.loss_largest_num_representable, model.loss_smallest_num_representable)

            # Optimizer Step, Propagate Loss backwards
            loss.backward()

            if quantize:
              
                # quantize gradients
                for name,param in model.named_parameters():
                    # Print BEFORE Gradients
                    # print(gradient)

                    param.grad.data = optimized_tensor_fp_quantize(param.grad.data, model.gradient_step_size, model.gradient_largest_num_representable, model.gradient_smallest_num_representable)
                    
                    # Print AFTER Gradients
                    # print(gradient)
                
            optimizer.step()

            # # TODO: Need to quantize the biases as well.
            if quantize:
              quantize_fp_model_weights(model)
              quantize_fp_model_biases(model)

            if (i + 1) % 600 == 0:
                print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                        % (epoch + 1, arg.epochs, i + 1,
                        len(train_set) // arg.batch_size, loss.data.item()))


# Gets accuracy given dataset as well as total test loss
def get_acc(model, criterion, test_loader, quantized=False):
    correct = 0
    total = 0

    for images, labels in test_loader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        testloss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        break

    if quantized:
      testloss = optimized_tensor_fp_quantize(testloss, model.loss_step_size, model.loss_largest_num_representable, model.loss_smallest_num_representable)

    return ((100 * correct / total), testloss)

## Quantization Model

In [56]:
# Assume that we always 
class MyConvNet_FIXED_POINT(nn.Module):
    def __init__(self, args, forwardfp, gradientfp, lossfp, weightsfp, biasfp): #xfp's are tuples of the # of bits in the fp rep & # of fractional bits
        super(MyConvNet_FIXED_POINT, self).__init__()


        ### gradient FP Parameters ###
        self.gradient_fp_bits = gradientfp[0]
        self.gradient_int_bits = (gradientfp[0] - 1 - gradientfp[1])
        self.gradient_fract_bits = gradientfp[1]
 
        self.gradient_step_size = 1 / np.power(2, gradientfp[1])
        self.gradient_largest_num_representable = (np.power(2, self.gradient_int_bits) - 1) + ((np.power(2, self.gradient_fract_bits)-1) * self.gradient_step_size)
        self.gradient_smallest_num_representable = -1 * np.power(2, self.gradient_int_bits)
        #########################

        ### forward FP Parameters ###
        self.forward_fp_bits = forwardfp[0]
        self.forward_int_bits = (forwardfp[0] - 1 - forwardfp[1])
        self.forward_fract_bits = forwardfp[1]
 
        self.forward_step_size = 1 / np.power(2, forwardfp[1])
        self.forward_largest_num_representable = (np.power(2, self.forward_int_bits) - 1) + ((np.power(2, self.forward_fract_bits)-1) * self.forward_step_size)
        self.forward_smallest_num_representable = -1 * np.power(2, self.forward_int_bits)
        #########################

        ### Loss FP Parameters ###
        self.loss_fp_bits = lossfp[0]
        self.loss_int_bits = (lossfp[0] - 1 - lossfp[1])
        self.loss_fract_bits = lossfp[1]

        self.loss_step_size = 1 / np.power(2, lossfp[1])
        self.loss_largest_num_representable = (np.power(2, self.loss_int_bits) - 1) + ((np.power(2, self.loss_fract_bits)-1) * self.loss_step_size)
        self.loss_smallest_num_representable = -1 * np.power(2, self.loss_int_bits)
        #########################

        ### weights FP Parameters ###
        self.weights_fp_bits = weightsfp[0]
        self.weights_int_bits = (weightsfp[0] - 1 - weightsfp[1])
        self.weights_fract_bits = weightsfp[1]
 
        self.weights_step_size = 1 / np.power(2, weightsfp[1])
        self.weights_largest_num_representable = (np.power(2, self.weights_int_bits) - 1) + ((np.power(2, self.weights_fract_bits)-1) * self.weights_step_size)
        self.weights_smallest_num_representable = -1 * np.power(2, self.weights_int_bits)
        #########################

        ### bias FP Parameters ###
        self.bias_fp_bits = biasfp[0]
        self.bias_int_bits = (biasfp[0] - 1 - biasfp[1])
        self.bias_fract_bits = biasfp[1]
 
        self.bias_step_size = 1 / np.power(2, biasfp[1])
        self.bias_largest_num_representable = (np.power(2, self.bias_int_bits) - 1) + ((np.power(2, self.bias_fract_bits)-1) * self.bias_step_size)
        self.bias_smallest_num_representable = -1 * np.power(2, self.bias_int_bits)
        #########################

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1   = nn.BatchNorm2d(16)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(32)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # Layer 3
        self.lin2  = nn.Linear(7*7*32, 10)

    # AUGMENT FORWARD PASS. forward pass not quantizes every single time we go through
    def forward(self, x):
        xq = optimized_tensor_fp_quantize(x, self.forward_step_size, self.forward_largest_num_representable, self.forward_smallest_num_representable)
        # Layer 1
        c1 = self.conv1(xq)
        c1q = optimized_tensor_fp_quantize(c1, self.forward_step_size, self.forward_largest_num_representable, self.forward_smallest_num_representable)
        b1  = self.bn1(c1q)
        b1q = optimized_tensor_fp_quantize(b1, self.forward_step_size, self.forward_largest_num_representable, self.forward_smallest_num_representable)
        a1  = self.act1(b1q)
        p1  = self.pool1(a1)
 
        # Layer 2
        c2  = self.conv2(p1)
        c2q = optimized_tensor_fp_quantize(c2, self.forward_step_size, self.forward_largest_num_representable, self.forward_smallest_num_representable)
        b2  = self.bn2(c2q)
        b2q = optimized_tensor_fp_quantize(b2, self.forward_step_size, self.forward_largest_num_representable, self.forward_smallest_num_representable)
        a2  = self.act2(b2q)
        p2  = self.pool2(a2)
 
        # Flatten and Layer 3
        flt = p2.view(p2.size(0), -1)
        out = self.lin2(flt)
        out_new = optimized_tensor_fp_quantize(out, self.forward_step_size, self.forward_largest_num_representable, self.forward_smallest_num_representable)
        return out

In [57]:
class QuantizedConvNet(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedConvNet, self).__init__()
        # QuantStub converts tensors from floating point to quantized.
        # This will only be used for inputs.
        self.quant = torch.quantization.QuantStub()
        # DeQuantStub converts tensors from quantized to floating point.
        # This will only be used for outputs.
        self.dequant = torch.quantization.DeQuantStub()
        # FP32 model
        self.model_fp32 = model_fp32

    def forward(self, x):
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.model_fp32(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x

# Test Code Here


In [58]:
# Hyper Parameter for FashionMNIST
input_size = 784
num_classes = 10



## Example of instantiating a model, training it with given hyperparameters, etc

In [59]:
args = easydict.EasyDict({
        "batch_size": 32,
        "epochs": 5,
        "lr": 0.001,
        "enable_cuda" : True,
        "L1norm" : False,
        "simpleNet" : True,
        "activation" : "relu", #relu, tanh, sigmoid
        "train_curve" : True, 
        "optimization" :"Adam"
})

# Dataset Loader (Input Pipeline)
quantize = True
train_loader = torch.utils.data.DataLoader(dataset = train_set, batch_size = args.batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = args.batch_size, shuffle = False)

num_bits_test = [64, 32, 16, 8, 4]
num_fractional_bits = [32, 16, 8, 4, 2]
acc = []

for (num_bits,num_fractional_bits) in zip(num_bits_test, num_fractional_bits):
  print(num_bits)
  print(num_fractional_bits)
  forwardfp= (num_bits, num_fractional_bits) 
  gradientfp= (num_bits, num_fractional_bits) 
  lossfp= (num_bits, num_fractional_bits) 
  weightsfp= (num_bits, num_fractional_bits) 
  biasfp=(num_bits, num_fractional_bits) 

  # Declare Model
  model = MyConvNet_FIXED_POINT(args, forwardfp, gradientfp, lossfp, weightsfp, biasfp).cuda()
  #model = MyConvNet_FP32(args).cuda()

  criterion = nn.CrossEntropyLoss().cuda()
  optimizer = torch.optim.Adam(model.parameters(), lr = args.lr) 

  # Training
  train_model(args, model, criterion, optimizer, train_loader, quantize)

  # Need to quantize the model afterwards
  quantize_fp_model_weights(model)

  # Testing ARC
  test_acc, test_loss = get_acc(model, criterion, test_loader, quantize)
  print("Test Accuracy: {}".format(test_acc))
  print("Test Loss: {}".format(test_loss))

  acc.append(test_acc.item())

plt.figure(0)
plt.plot(num_bits_test, acc)
plt.title("Varying Number of Quantized Bits")
plt.xlabel("Number of Quantized Bits")
plt.ylabel("Acc (%)")
plt.show()


64
32
---Training started
Epoch: [ 1/ 5], Step: [ 600/ 1875], Loss: 0.3993
Epoch: [ 1/ 5], Step: [ 1200/ 1875], Loss: 0.4763


KeyboardInterrupt: ignored

In [62]:
forwardfp= (16, 8) 
gradientfp= (16, 10) 
lossfp= (16, 10) 
weightsfp= (32, 12) 
biasfp=(64, 32) 

# Declare Model
model = MyConvNet_FIXED_POINT(args, forwardfp, gradientfp, lossfp, weightsfp, biasfp).cuda()
#model = MyConvNet_FP32(args).cuda()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = args.lr) 

# Training
train_model(args, model, criterion, optimizer, train_loader, quantize)

# Need to quantize the model afterwards
quantize_fp_model_weights(model)

# Testing ARC
test_acc, test_loss = get_acc(model, criterion, test_loader, quantize)
print("Test Accuracy: {}".format(test_acc))
print("Test Loss: {}".format(test_loss))


---Training started
Epoch: [ 1/ 5], Step: [ 600/ 1875], Loss: 0.6924
Epoch: [ 1/ 5], Step: [ 1200/ 1875], Loss: 0.7031
Epoch: [ 1/ 5], Step: [ 1800/ 1875], Loss: 0.1729
Epoch: [ 2/ 5], Step: [ 600/ 1875], Loss: 0.2891
Epoch: [ 2/ 5], Step: [ 1200/ 1875], Loss: 0.2002
Epoch: [ 2/ 5], Step: [ 1800/ 1875], Loss: 0.4658
Epoch: [ 3/ 5], Step: [ 600/ 1875], Loss: 0.2021
Epoch: [ 3/ 5], Step: [ 1200/ 1875], Loss: 0.3037
Epoch: [ 3/ 5], Step: [ 1800/ 1875], Loss: 0.0977
Epoch: [ 4/ 5], Step: [ 600/ 1875], Loss: 0.1113
Epoch: [ 4/ 5], Step: [ 1200/ 1875], Loss: 0.4414
Epoch: [ 4/ 5], Step: [ 1800/ 1875], Loss: 0.1992
Epoch: [ 5/ 5], Step: [ 600/ 1875], Loss: 0.3311
Epoch: [ 5/ 5], Step: [ 1200/ 1875], Loss: 0.1680
Epoch: [ 5/ 5], Step: [ 1800/ 1875], Loss: 0.1729
Test Accuracy: 93.75
Test Loss: 0.359375


In [60]:
fp_32model = MyConvNet_FP32(args).cuda()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(fp_32model.parameters(), lr = args.lr) 

train_model(args, fp_32model, criterion, optimizer, train_loader, False)

test_acc, test_loss = get_acc(fp_32model, criterion, test_loader, False)
print("Test Accuracy: {}".format(test_acc))
print("Test Loss: {}".format(test_loss))

---Training started
Epoch: [ 1/ 5], Step: [ 600/ 1875], Loss: 0.3771
Epoch: [ 1/ 5], Step: [ 1200/ 1875], Loss: 0.4219
Epoch: [ 1/ 5], Step: [ 1800/ 1875], Loss: 0.3681
Epoch: [ 2/ 5], Step: [ 600/ 1875], Loss: 0.3782
Epoch: [ 2/ 5], Step: [ 1200/ 1875], Loss: 0.3302
Epoch: [ 2/ 5], Step: [ 1800/ 1875], Loss: 0.4099
Epoch: [ 3/ 5], Step: [ 600/ 1875], Loss: 0.5968
Epoch: [ 3/ 5], Step: [ 1200/ 1875], Loss: 0.2400
Epoch: [ 3/ 5], Step: [ 1800/ 1875], Loss: 0.3695
Epoch: [ 4/ 5], Step: [ 600/ 1875], Loss: 0.2213
Epoch: [ 4/ 5], Step: [ 1200/ 1875], Loss: 0.1097
Epoch: [ 4/ 5], Step: [ 1800/ 1875], Loss: 0.3762
Epoch: [ 5/ 5], Step: [ 600/ 1875], Loss: 0.0675
Epoch: [ 5/ 5], Step: [ 1200/ 1875], Loss: 0.2744
Epoch: [ 5/ 5], Step: [ 1800/ 1875], Loss: 0.1331
Test Accuracy: 87.5
Test Loss: 0.6163979172706604


## Preston's Training of Model

In [ ]:
# cuda_device = torch.device("cuda:0")
# cpu_device = torch.device("cpu:0")

# # Initial model
# FP_model = MyConvNet_FP32(args).cuda()
# FP_fused_model = MyConvNet_FP32(args)

# # Training stuff
# criterion = nn.CrossEntropyLoss().cuda()
# optimizer = torch.optim.SGD(FP_model.parameters(), lr = learning_rate) 

# train_model(FP_model, criterion, optimizer, train_loader)



In [ ]:
# test_acc_FP, test_loss_FP = get_acc(FP_model, criterion, test_loader)
# print("Test Accuracy: {}".format(test_acc_FP))
# print("Test Loss: {}".format(test_loss_FP))


In [ ]:

# copy_models(FP_model,FP_fused_model)   
# fuse_conv_bn(FP_model,FP_fused_model)

# quantized_model = QuantizedConvNet(model_fp32=FP_fused_model).to(cpu_device)

# quantization_config = torch.quantization.get_default_qconfig("fbgemm")
# quantized_model.qconfig = quantization_config
# torch.quantization.prepare_qat(quantized_model, inplace=True)

# quantized_model.to(cuda_device)
# train_model(quantized_model, criterion, optimizer, train_loader)
# # quantized_model.to(cpu_device)

# # quantized_model = torch.quantization.convert(quantized_model, inplace=True)



In [ ]:


# test_acc, test_loss = get_acc(quantized_model, criterion, test_loader)
# print("Test Accuracy: {}".format(test_acc))
# print("Test Loss: {}".format(test_loss))

